In [ ]:
import os

import numpy as np
import pandas as pd

os.sys.path.append(os.path.abspath('../src'))
from data import loader
from data import preprocessor

mapname = 'Town01'

### Running

In [ ]:
carla_data, sumo_data = loader.load_synthetic_data(os.path.abspath(f'../data/synthetic/{mapname}'))
drivers = ['D1', 'D2', 'D3', 'D4', 'D5'] # List of drivers to load
uah_data = loader.read_data(drivers, os.path.abspath('../data/base/UAH-DRIVESET-v1'))

In [14]:
merged_path = f'../data/merged/{mapname}'
merged_sumo_uah, merged_carla_uah = loader.load_merged_data(merged_path)

In [ ]:
X_normal, Y_normal = preprocessor.sliding_windows(merged_sumo_uah['fixed']['normal'], 'normal', 10, 5)
X_aggressive, Y_aggressive = preprocessor.sliding_windows(merged_sumo_uah['fixed']['aggressive'], 'aggressive', 10, 5)
X = pd.concat([X_normal, X_aggressive], ignore_index=True)
y = pd.concat([Y_normal, Y_aggressive], ignore_index=True)

In [29]:
merged_sumo_uah['fixed']['normal'].shape

(124919, 4)

In [27]:
teste1.shape

(24982, 40)